# `dkmri.py` example

Let's start by importing the packages and modules used in this notebook.

In [1]:
import dkmri
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np

Then, let's load a dataset to work on. Replace the code below with code that loads your data.

In [2]:
data = nib.load("data.nii.gz").get_fdata()
bvals = np.loadtxt("data.bval")
bvecs = np.loadtxt("data.bvec").T
mask = nib.load("mask.nii.gz").get_fdata().astype(bool)

The diffusion kurtosis imaging signal model can be fit to data using the function `dkmri.fit`. The docstring contains important information about the function:

In [3]:
dkmri.fit?

Signature: dkmri.fit(data, bvals, bvecs, mask=None, alpha=None, seed=123, quiet=False)
Docstring:
Estimate diffusion and kurtosis tensor elements from data.

This function does the following:

    1. Remove infinities, nans, and negative values in `data`, and scale
       values in `data` and `bvals`.
    2. Estimate model parameters using standard NLLS.
    3. Train multilayer perceptrons to predict kurtosis measures from data
       in voxels where the apparent kurtosis coefficient computed from the
       NLLS fit results is non-negative along all directions.
    4. Estimate model parameters using regularized NLLS where the
       regularization terms increase the objective function value when
       MK, AK, and RK deviate from their predicted values. Axially
       symmetric tensors with plausible magnitudes are used as initial
       positions for the fit.

Parameters
----------
data : numpy.ndarray
    Floating-point array with shape (..., number of acquisitions).
bvals : numpy.n

Let's fit the model to data.

In [4]:
fit_result = dkmri.fit(data, bvals, bvecs, mask)

Fitting DKI to data with standard NLLS


`dkmri.fit` returns a `FitResult` object whose attributes contain the results of the fit.

In [ ]:
fit_result?

Parameter maps can be computed using the followin functions: 

- `dkmri.params_to_ad`
- `dkmri.params_to_ak`
- `dkmri.params_to_fa`
- `dkmri.params_to_md`
- `dkmri.params_to_mk`
- `dkmri.params_to_rd`
- `dkmri.params_to_rk`

In addition, we can use the function `dkmri.signal` to compute the signal from the estimated parameters.

Let's compute and show some parameter maps.

In [ ]:
z = 40

mk = dkmri.params_to_mk(fit_result.params, mask)
plt.imshow(np.rot90(mk[..., z]), vmin=0, vmax=2, cmap="gray")
plt.colorbar()
plt.title("Mean kurtosis")
plt.xticks([])
plt.yticks([])
plt.show()

ak = dkmri.params_to_ak(fit_result.params, mask)
plt.imshow(np.rot90(ak[..., z]), vmin=0, vmax=1.5, cmap="gray")
plt.colorbar()
plt.title("Axial kurtosis")
plt.xticks([])
plt.yticks([])
plt.show()

rk = dkmri.params_to_rk(fit_result.params, mask)
plt.imshow(np.rot90(rk[..., z]), vmin=0, vmax=4, cmap="gray")
plt.colorbar()
plt.title("Radial kurtosis")
plt.xticks([])
plt.yticks([])
plt.show()